In [39]:
!pip3 install -q kagglehub

from pathlib import Path
import os, json, gc
import numpy as np
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip


In [40]:
DATASET_ID = "computingvictor/transactions-fraud-datasets"
DATASET_FOLDER_NAME = "transactions-fraud-datasets"

N_CLIENTS = 300
SEQ_LEN = 10
MAX_WINDOWS_TRAIN = 50_000
MAX_WINDOWS_VAL   = 20_000
MAX_WINDOWS_TEST  = 20_000

BATCH_SIZE = 128
EPOCHS = 15
LR = 1e-3

POS_WEIGHT_CAP = 50.0
SEED = 0
# =====================

np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

device: cpu


In [41]:
import kagglehub

def find_repo_root():
    cur = Path.cwd().resolve()
    for p in [cur] + list(cur.parents):
        if (p / ".git").exists():
            return p
    return cur

def resolve_dataset_dir():
    env = os.getenv("FRAUD_DATA_DIR")
    if env:
        p = Path(env).expanduser().resolve()
        if p.exists():
            return p

    repo_root = find_repo_root()
    local = repo_root / "data" / DATASET_FOLDER_NAME
    if local.exists():
        return local.resolve()

    print("Dataset not found locally - downloading via kagglehub...")
    return Path(kagglehub.dataset_download(DATASET_ID)).resolve()

def parse_amount(x):
    x = str(x).replace("$","").replace(",","").strip()
    try:
        return float(x)
    except:
        return np.nan

In [42]:
# Load Labels + Transactions Subset

dataset_dir = resolve_dataset_dir()
print("Using dataset directory:", dataset_dir)

tx_path = dataset_dir / "transactions_data.csv"
labels_path = dataset_dir / "train_fraud_labels.json"

assert tx_path.exists(), f"Missing {tx_path}"
assert labels_path.exists(), f"Missing {labels_path}"

with open(labels_path, "r") as f:
    labels_raw = json.load(f)

target_map = labels_raw["target"]
labels = pd.DataFrame({
    "transaction_id": list(target_map.keys()),
    "target": [1 if v == "Yes" else 0 for v in target_map.values()]
})
labels["transaction_id"] = labels["transaction_id"].astype(str)
labels["target"] = labels["target"].astype(int)

usecols = ["id","date","client_id","card_id","amount","use_chip","merchant_state","mcc","errors"]

# choose clients from an early slice
first_chunk = pd.read_csv(tx_path, usecols=["client_id"], nrows=200_000)
clients = first_chunk["client_id"].dropna().unique()

chosen_clients = set(np.random.choice(clients, size=min(N_CLIENTS, len(clients)), replace=False))

chunks = []
for chunk in pd.read_csv(tx_path, usecols=usecols, chunksize=200_000):
    keep = chunk[chunk["client_id"].isin(chosen_clients)]
    if len(keep):
        chunks.append(keep)

tx = pd.concat(chunks, ignore_index=True)
tx["id"] = tx["id"].astype(str)

tx = tx.merge(labels, left_on="id", right_on="transaction_id", how="left").drop(columns=["transaction_id"])

print("Rows:", len(tx))
print("Labeled rows:", tx["target"].notna().sum())
tx.head()

Dataset not found locally - downloading via kagglehub...
Using dataset directory: /Users/anissasoungpanya/.cache/kagglehub/datasets/computingvictor/transactions-fraud-datasets/versions/1
Rows: 3358392
Labeled rows: 2249957


,id,date,client_id,card_id,amount,use_chip,merchant_state,mcc,errors,target
0,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,IA,5311,NaN,0.0
1,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,CA,4829,NaN,0.0
2,7475333,2010-01-01 00:07:00,1807,165,$4.81,Swipe Transaction,NY,5942,NaN,0.0
3,7475337,2010-01-01 00:21:00,351,1112,$10.74,Swipe Transaction,NY,5813,NaN,NaN
4,7475344,2010-01-01 00:32:00,646,2093,$73.79,Swipe Transaction,PA,7538,NaN,0.0


In [43]:
# Clean + Basic Feature Columns

tx["date"] = pd.to_datetime(tx["date"], errors="coerce")
tx = tx.dropna(subset=["date"])

tx["amount"] = tx["amount"].apply(parse_amount)
tx = tx.dropna(subset=["amount"])

tx["errors"] = tx["errors"].fillna("None").astype(str)
tx["use_chip"] = tx["use_chip"].fillna("Unknown").astype(str)
tx["merchant_state"] = tx["merchant_state"].fillna("Unknown").astype(str)
tx["mcc"] = tx["mcc"].fillna(-1).astype(int).astype(str)

tx = tx.sort_values(["client_id","date"]).reset_index(drop=True)

tx["hour"] = tx["date"].dt.hour
tx["dayofweek"] = tx["date"].dt.dayofweek

# time-delta / "velocity" features (using only past info)
tx["delta_t_client_sec"] = tx.groupby("client_id")["date"].diff().dt.total_seconds()
tx["delta_t_card_sec"]   = tx.groupby("card_id")["date"].diff().dt.total_seconds()

# fill missing deltas (first transaction per group) with median delta
for col in ["delta_t_client_sec", "delta_t_card_sec"]:
    tx[col] = tx[col].fillna(tx[col].median())
    # cap extreme gaps to reduce outlier impact
    cap = tx[col].quantile(0.99)
    tx[col] = tx[col].clip(lower=0, upper=cap)

# log transform to compress scale (more stable for models)
tx["log_delta_t_client"] = np.log1p(tx["delta_t_client_sec"])
tx["log_delta_t_card"]   = np.log1p(tx["delta_t_card_sec"])

tx.head()

,id,date,client_id,card_id,amount,use_chip,merchant_state,mcc,errors,target,hour,dayofweek,delta_t_client_sec,delta_t_card_sec,log_delta_t_client,log_delta_t_card
0,7477094,2010-01-01 11:58:00,1,4652,15.09,Swipe Transaction,FL,4121,None,0.0,11,4,13140.0,41640.0,9.483492,10.636841
1,7477168,2010-01-01 12:11:00,1,3682,6.01,Swipe Transaction,FL,5813,None,0.0,12,4,780.0,41640.0,6.660575,10.636841
2,7477216,2010-01-01 12:18:00,1,3682,14.58,Online Transaction,Unknown,4121,None,0.0,12,4,420.0,420.0,6.042633,6.042633
3,7477978,2010-01-01 15:09:00,1,4652,14.66,Online Transaction,Unknown,4121,None,0.0,15,4,10260.0,11460.0,9.236106,9.346705
4,7478279,2010-01-01 16:26:00,1,4652,22.77,Swipe Transaction,FL,4121,None,NaN,16,4,4620.0,4620.0,8.438366,8.438366


In [44]:
def time_split(df, client_col="client_id", frac_train=0.70, frac_val=0.15, min_len=20):
    train_idx, val_idx, test_idx = [], [], []
    for cid, g in df.groupby(client_col, sort=False):
        n = len(g)
        if n < min_len:
            continue
        t1 = int(n * frac_train)
        t2 = int(n * (frac_train + frac_val))
        idx = g.index.to_numpy()
        train_idx.append(idx[:t1])
        val_idx.append(idx[t1:t2])
        test_idx.append(idx[t2:])
    return np.concatenate(train_idx), np.concatenate(val_idx), np.concatenate(test_idx)

train_idx, val_idx, test_idx = time_split(tx)

train_df = tx.loc[train_idx].copy()
val_df   = tx.loc[val_idx].copy()
test_df  = tx.loc[test_idx].copy()

print(len(train_df), len(val_df), len(test_df))

2350739 503758 503895


In [45]:
# Encode + Scale

num_cols = ["amount", "hour", "dayofweek", "log_delta_t_client", "log_delta_t_card"]
cat_cols = ["use_chip", "merchant_state", "mcc", "errors"]

enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
scaler = StandardScaler()

Xtr_cat = enc.fit_transform(train_df[cat_cols])
Xva_cat = enc.transform(val_df[cat_cols])
Xte_cat = enc.transform(test_df[cat_cols])

Xtr_num = scaler.fit_transform(train_df[num_cols])
Xva_num = scaler.transform(val_df[num_cols])
Xte_num = scaler.transform(test_df[num_cols])

X_train = np.hstack([Xtr_num, Xtr_cat]).astype(np.float32)
X_val   = np.hstack([Xva_num, Xva_cat]).astype(np.float32)
X_test  = np.hstack([Xte_num, Xte_cat]).astype(np.float32)

y_train = train_df["target"].to_numpy()
y_val   = val_df["target"].to_numpy()
y_test  = test_df["target"].to_numpy()

X_train.shape

(2350739, 9)

In [46]:
def build_sequences_sampled(df_part, X_part, y_part, seq_len=10, max_windows=50_000, client_col="client_id", per_client_cap=50, seed=0):
    X_seqs = np.zeros((max_windows, seq_len, X_part.shape[1]), dtype=np.float32)
    y_seqs = np.zeros((max_windows,), dtype=np.int64)

    k = 0
    start = 0
    rng = np.random.default_rng(seed)

    for cid, g in df_part.groupby(client_col, sort=False):
        n = len(g)
        if n <= seq_len:
            start += n
            continue

        Xg = X_part[start:start+n]
        yg = y_part[start:start+n]

        valid_t = np.where(~np.isnan(yg))[0]
        valid_t = valid_t[valid_t >= seq_len]
        if len(valid_t) == 0:
            start += n
            continue

        take = min(len(valid_t), per_client_cap)
        chosen = rng.choice(valid_t, size=take, replace=False)

        for t in chosen:
            if k >= max_windows:
                return X_seqs[:k], y_seqs[:k]
            X_seqs[k] = Xg[t-seq_len:t]
            y_seqs[k] = int(yg[t])
            k += 1

        start += n

    return X_seqs[:k], y_seqs[:k]


Xtr_seq, ytr_seq = build_sequences_sampled(train_df, X_train, y_train, seq_len=SEQ_LEN, max_windows=MAX_WINDOWS_TRAIN, seed=SEED)
Xva_seq, yva_seq = build_sequences_sampled(val_df,   X_val,   y_val,   seq_len=SEQ_LEN, max_windows=MAX_WINDOWS_VAL, seed=SEED)
Xte_seq, yte_seq = build_sequences_sampled(test_df,  X_test,  y_test,  seq_len=SEQ_LEN, max_windows=MAX_WINDOWS_TEST, seed=SEED)

print("Train windows:", Xtr_seq.shape, "fraud rate:", ytr_seq.mean(), "count:", int(ytr_seq.sum()))
print("Val windows:",   Xva_seq.shape, "fraud rate:", yva_seq.mean(), "count:", int(yva_seq.sum()))
print("Test windows:",  Xte_seq.shape, "fraud rate:", yte_seq.mean(), "count:", int(yte_seq.sum()))

Train windows: (15000, 10, 9) fraud rate: 0.0018 count: 27
Val windows: (15000, 10, 9) fraud rate: 0.001 count: 15
Test windows: (15000, 10, 9) fraud rate: 0.0018 count: 27


In [47]:
def compute_metrics(y_true, y_prob, threshold=0.5):
    y_true = np.asarray(y_true).astype(int)
    y_prob = np.asarray(y_prob)

    y_pred = (y_prob >= threshold).astype(int)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    fpr = ((y_pred == 1) & (y_true == 0)).sum() / max((y_true == 0).sum(), 1)

    roc = roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else np.nan
    pr  = average_precision_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else np.nan
    return {"precision": prec, "recall": rec, "f1": f1, "fpr": fpr, "roc_auc": roc, "pr_auc": pr}

def best_threshold_f1(y_true, y_prob, thresholds=None, min_pred_pos=1):
    """
    Picks threshold that maximizes F1 on validation, but avoids degenerate thresholds
    that predict 0 positives (or fewer than min_pred_pos).

    Uses quantiles of y_prob as candidate thresholds by default (more stable than fixed linspace).
    """
    y_true = np.asarray(y_true).astype(int)
    y_prob = np.asarray(y_prob)

    if thresholds is None:
        qs = np.linspace(0.01, 0.99, 300)
        thresholds = np.unique(np.quantile(y_prob, qs))

    best_t, best_f1, best_m = 0.5, -1, None
    for t in thresholds:
        y_pred = (y_prob >= t).astype(int)
        if y_pred.sum() < min_pred_pos:
            continue
        m = compute_metrics(y_true, y_prob, threshold=float(t))
        if m["f1"] > best_f1:
            best_f1, best_t, best_m = m["f1"], float(t), m

    if best_m is None:
        best_t, best_m = 0.5, compute_metrics(y_true, y_prob, threshold=0.5)

    return best_t, best_m

In [48]:
# Sklearn Baselines

Xtr_flat = Xtr_seq.reshape(Xtr_seq.shape[0], -1)
Xva_flat = Xva_seq.reshape(Xva_seq.shape[0], -1)
Xte_flat = Xte_seq.reshape(Xte_seq.shape[0], -1)

ytr = ytr_seq.astype(np.int64)
yva = yva_seq.astype(np.int64)
yte = yte_seq.astype(np.int64)

baseline_results = {}

# Logistic Regression
logreg = LogisticRegression(max_iter=2000, class_weight="balanced", n_jobs=-1)
logreg.fit(Xtr_flat, ytr)

va_prob_lr = logreg.predict_proba(Xva_flat)[:, 1]
best_t_lr, va_metrics_lr = best_threshold_f1(yva, va_prob_lr)
te_prob_lr = logreg.predict_proba(Xte_flat)[:, 1]
te_metrics_lr = compute_metrics(yte, te_prob_lr, threshold=best_t_lr)

baseline_results["LogReg"] = {"best_t": best_t_lr, "val": va_metrics_lr, "test": te_metrics_lr}
print("LogReg:", baseline_results["LogReg"])

# Random Forest
rf = RandomForestClassifier(n_estimators=300, min_samples_leaf=2, class_weight="balanced_subsample", n_jobs=-1, random_state=SEED)
rf.fit(Xtr_flat, ytr)

va_prob_rf = rf.predict_proba(Xva_flat)[:, 1]
best_t_rf, va_metrics_rf = best_threshold_f1(yva, va_prob_rf)
te_prob_rf = rf.predict_proba(Xte_flat)[:, 1]
te_metrics_rf = compute_metrics(yte, te_prob_rf, threshold=best_t_rf)

baseline_results["RandomForest"] = {"best_t": best_t_rf, "val": va_metrics_rf, "test": te_metrics_rf}
print("RandomForest:", baseline_results["RandomForest"])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogReg: {'best_t': 0.9989585424887892, 'val': {'precision': 0.006666666666666667, 'recall': 0.06666666666666667, 'f1': 0.012121212121212121, 'fpr': np.float64(0.009943276609943277), 'roc_auc': 0.5033611389166945, 'pr_auc': 0.002445907147746244}, 'test': {'precision': 0.006060606060606061, 'recall': 0.037037037037037035, 'f1': 0.010416666666666666, 'fpr': np.float64(0.010953048821211514), 'roc_auc': 0.5628699560443365, 'pr_auc': 0.007005624797213152}}
RandomForest: {'best_t': 0.0033271111111111114, 'val': {'precision': 0.0014099400775467042, 'recall': 0.5333333333333333, 'f1': 0.0028124450694322375, 'fpr': np.float64(0.3781114447781114), 'roc_auc': 0.5333044155266378, 'pr_auc': 0.0010992016488155852}, 'test': {'precision': 0.0010783608914450035, 'recall': 0.2222222222222222, 'f1': 0.0021463065641209086, 'fpr': np.float64(0.37120149602618047), 'roc_auc': 0.43351489471171567, 'pr_auc': 0.001741175698951547}}


In [49]:
# Torch Dataset + Loaders

class SeqDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y).float()
    def __len__(self):
        return len(self.y)
    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_loader = DataLoader(SeqDataset(Xtr_seq, ytr_seq), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(SeqDataset(Xva_seq, yva_seq), batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(SeqDataset(Xte_seq, yte_seq), batch_size=BATCH_SIZE, shuffle=False)

In [50]:
# Model Definitions (CNN, CNN-RNN)

class FraudCNNRNN(nn.Module):
    def __init__(self, num_features, conv_channels=64, kernel_size=3, hidden_size=64, num_layers=1, dropout=0.1):
        super().__init__()
        padding = kernel_size // 2
        self.conv = nn.Sequential(
            nn.Conv1d(num_features, conv_channels, kernel_size, padding=padding),
            nn.BatchNorm1d(conv_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(conv_channels, conv_channels, kernel_size, padding=padding),
            nn.BatchNorm1d(conv_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.rnn = nn.LSTM(
            input_size=conv_channels,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=(dropout if num_layers > 1 else 0.0),
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = x.transpose(1, 2)      # (B,T,F)->(B,F,T)
        x = self.conv(x)           # (B,C,T)
        x = x.transpose(1, 2)      # (B,T,C)
        _, (h_n, _) = self.rnn(x)
        h_last = h_n[-1]
        return self.fc(h_last).squeeze(1)

class FraudCNNBaseline(nn.Module):
    def __init__(self, num_features, conv_channels=64, kernel_size=3, dropout=0.1, pool="max"):
        super().__init__()
        padding = kernel_size // 2
        self.conv = nn.Sequential(
            nn.Conv1d(num_features, conv_channels, kernel_size, padding=padding),
            nn.BatchNorm1d(conv_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(conv_channels, conv_channels, kernel_size, padding=padding),
            nn.BatchNorm1d(conv_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        assert pool in ("max", "avg")
        self.pool = pool
        self.fc = nn.Linear(conv_channels, 1)

    def forward(self, x):
        x = x.transpose(1, 2)      # (B,T,F)->(B,F,T)
        x = self.conv(x)           # (B,C,T)
        x = x.max(dim=2).values if self.pool == "max" else x.mean(dim=2)  # (B,C)
        return self.fc(x).squeeze(1)

class FraudLSTM(nn.Module):
    def __init__(self, num_features, hidden_size=64, num_layers=1, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=(dropout if num_layers > 1 else 0.0),
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)     # h_n: (num_layers, B, hidden)
        h_last = h_n[-1]               # (B, hidden)
        return self.fc(h_last).squeeze(1)


In [51]:
# Shared Torch Train/Eval Functions

def get_pos_weight(y_np, cap=POS_WEIGHT_CAP):
    pos = float(np.sum(y_np))
    neg = float(len(y_np) - pos)
    raw = neg / max(pos, 1.0)
    return torch.tensor([min(raw, cap)], dtype=torch.float32, device=device)

def eval_pr_auc_torch(model, loader):
    model.eval()
    all_y, all_p = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            prob = torch.sigmoid(model(xb)).cpu().numpy()
            all_p.append(prob)
            all_y.append(yb.numpy())
    y_true = np.concatenate(all_y).astype(int)
    y_prob = np.concatenate(all_p)
    return average_precision_score(y_true, y_prob)

def get_probs_from_loader(model, loader):
    model.eval()
    all_y, all_p = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            prob = torch.sigmoid(model(xb)).cpu().numpy()
            all_p.append(prob)
            all_y.append(yb.numpy())
    return np.concatenate(all_y).astype(int), np.concatenate(all_p)

def train_torch_model(model, train_loader, val_loader, epochs=EPOCHS, lr=LR):
    model = model.to(device)
    pos_weight = get_pos_weight(ytr_seq, cap=POS_WEIGHT_CAP)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_pr, best_state = -1, None

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0.0

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * len(yb)

        val_pr = eval_pr_auc_torch(model, val_loader)
        print(f"Epoch {epoch:02d} | loss={total_loss/len(train_loader.dataset):.4f} | val_pr_auc={val_pr:.6f}")

        if val_pr > best_pr:
            best_pr = val_pr
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

    if best_state is not None:
        model.load_state_dict(best_state)

    return model

In [52]:
# Train CNN-RNN

cnn_rnn = FraudCNNRNN(num_features=Xtr_seq.shape[2], conv_channels=64, kernel_size=3, hidden_size=64, num_layers=1, dropout=0.1)
cnn_rnn = train_torch_model(cnn_rnn, train_loader, val_loader, epochs=EPOCHS, lr=LR)

Epoch 01 | loss=0.3233 | val_pr_auc=0.001026
Epoch 02 | loss=0.2875 | val_pr_auc=0.002045
Epoch 03 | loss=0.2645 | val_pr_auc=0.003059
Epoch 04 | loss=0.2542 | val_pr_auc=0.001861
Epoch 05 | loss=0.2511 | val_pr_auc=0.004133
Epoch 06 | loss=0.2377 | val_pr_auc=0.003004
Epoch 07 | loss=0.2155 | val_pr_auc=0.003726
Epoch 08 | loss=0.2169 | val_pr_auc=0.001819
Epoch 09 | loss=0.1910 | val_pr_auc=0.002116
Epoch 10 | loss=0.2020 | val_pr_auc=0.004246
Epoch 11 | loss=0.1816 | val_pr_auc=0.004045
Epoch 12 | loss=0.1751 | val_pr_auc=0.002523
Epoch 13 | loss=0.1472 | val_pr_auc=0.002417
Epoch 14 | loss=0.1730 | val_pr_auc=0.001934
Epoch 15 | loss=0.1537 | val_pr_auc=0.001762


In [53]:
# Train CNN Baseline

cnn = FraudCNNBaseline(num_features=Xtr_seq.shape[2], conv_channels=64, kernel_size=3, dropout=0.1, pool="max")
cnn = train_torch_model(cnn, train_loader, val_loader, epochs=EPOCHS, lr=LR)

Epoch 01 | loss=0.4033 | val_pr_auc=0.001374
Epoch 02 | loss=0.2873 | val_pr_auc=0.002542
Epoch 03 | loss=0.2646 | val_pr_auc=0.002740
Epoch 04 | loss=0.2532 | val_pr_auc=0.002118
Epoch 05 | loss=0.2388 | val_pr_auc=0.002211
Epoch 06 | loss=0.2211 | val_pr_auc=0.002051
Epoch 07 | loss=0.2120 | val_pr_auc=0.001150
Epoch 08 | loss=0.2286 | val_pr_auc=0.001166
Epoch 09 | loss=0.2021 | val_pr_auc=0.001108
Epoch 10 | loss=0.1868 | val_pr_auc=0.001518
Epoch 11 | loss=0.1724 | val_pr_auc=0.001189
Epoch 12 | loss=0.1560 | val_pr_auc=0.001752
Epoch 13 | loss=0.1470 | val_pr_auc=0.000948
Epoch 14 | loss=0.1388 | val_pr_auc=0.001902
Epoch 15 | loss=0.1436 | val_pr_auc=0.001241


In [54]:
# Train LSTM

lstm = FraudLSTM(num_features=Xtr_seq.shape[2], hidden_size=64, num_layers=1, dropout=0.1)
lstm = train_torch_model(lstm, train_loader, val_loader, epochs=EPOCHS, lr=LR)

Epoch 01 | loss=0.3581 | val_pr_auc=0.001171
Epoch 02 | loss=0.2802 | val_pr_auc=0.001274
Epoch 03 | loss=0.2779 | val_pr_auc=0.006333
Epoch 04 | loss=0.2752 | val_pr_auc=0.011796
Epoch 05 | loss=0.2809 | val_pr_auc=0.007345
Epoch 06 | loss=0.2733 | val_pr_auc=0.005837
Epoch 07 | loss=0.2732 | val_pr_auc=0.008158
Epoch 08 | loss=0.2691 | val_pr_auc=0.003481
Epoch 09 | loss=0.2645 | val_pr_auc=0.016433
Epoch 10 | loss=0.2642 | val_pr_auc=0.004963
Epoch 11 | loss=0.2632 | val_pr_auc=0.012358
Epoch 12 | loss=0.2549 | val_pr_auc=0.002657
Epoch 13 | loss=0.2542 | val_pr_auc=0.004350
Epoch 14 | loss=0.2585 | val_pr_auc=0.003084
Epoch 15 | loss=0.2513 | val_pr_auc=0.002932


In [55]:
# Evaluate All Models + Build Final Table

rows = []

def add_row(name, split, best_t, metrics):
    rows.append({"model": name, "split": split, "best_threshold_from_val": best_t, **metrics})

# sklearn baselines
for name, info in baseline_results.items():
    add_row(name, "val",  info["best_t"], info["val"])
    add_row(name, "test", info["best_t"], info["test"])

# CNN-RNN
yva_true, yva_prob = get_probs_from_loader(cnn_rnn, val_loader)
best_t, va_m = best_threshold_f1(yva_true, yva_prob)
yte_true, yte_prob = get_probs_from_loader(cnn_rnn, test_loader)
te_m = compute_metrics(yte_true, yte_prob, threshold=best_t)
add_row("CNN-RNN", "val", best_t, va_m)
add_row("CNN-RNN", "test", best_t, te_m)

# CNN
yva_true, yva_prob = get_probs_from_loader(cnn, val_loader)
best_t, va_m = best_threshold_f1(yva_true, yva_prob)
yte_true, yte_prob = get_probs_from_loader(cnn, test_loader)
te_m = compute_metrics(yte_true, yte_prob, threshold=best_t)
add_row("CNN", "val", best_t, va_m)
add_row("CNN", "test", best_t, te_m)

# LSTM
yva_true, yva_prob = get_probs_from_loader(lstm, val_loader)
best_t, va_m = best_threshold_f1(yva_true, yva_prob)
yte_true, yte_prob = get_probs_from_loader(lstm, test_loader)
te_m = compute_metrics(yte_true, yte_prob, threshold=best_t)
add_row("LSTM", "val", best_t, va_m)
add_row("LSTM", "test", best_t, te_m)

df_compare = pd.DataFrame(rows)
df_compare = df_compare[[
    "model","split","best_threshold_from_val",
    "precision","recall","f1","fpr","roc_auc","pr_auc"
]].sort_values(["model","split"])

df_compare

,model,split,best_threshold_from_val,precision,recall,f1,fpr,roc_auc,pr_auc
7,CNN,test,0.114856,0.003628,0.185185,0.007117,0.091698,0.609114,0.003040
6,CNN,val,0.114856,0.004348,0.400000,0.008602,0.091692,0.685063,0.002740
5,CNN-RNN,test,0.467768,0.010101,0.111111,0.018519,0.019635,0.530352,0.004215
4,CNN-RNN,val,0.467768,0.010067,0.200000,0.019169,0.019686,0.659148,0.004246
9,LSTM,test,0.297354,0.009390,0.074074,0.016667,0.014092,0.426899,0.002579
8,LSTM,val,0.297354,0.015000,0.200000,0.027907,0.013146,0.546894,0.016433
1,LogReg,test,0.998959,0.006061,0.037037,0.010417,0.010953,0.562870,0.007006
0,LogReg,val,0.998959,0.006667,0.066667,0.012121,0.009943,0.503361,0.002446
3,RandomForest,test,0.003327,0.001078,0.222222,0.002146,0.371201,0.433515,0.001741
2,RandomForest,val,0.003327,0.001410,0.533333,0.002812,0.378111,0.533304,0.001099


In [56]:
print("train fraud rate:", ytr_seq.mean(), "count:", int(ytr_seq.sum()), "/", len(ytr_seq))
print("val fraud rate:",   yva_seq.mean(), "count:", int(yva_seq.sum()), "/", len(yva_seq))
print("test fraud rate:",  yte_seq.mean(), "count:", int(yte_seq.sum()), "/", len(yte_seq))

fraud_by_client = tx.groupby("client_id")["target"].sum(min_count=1)
print("clients with >=1 fraud:", int((fraud_by_client > 0).sum()), "/", fraud_by_client.shape[0])

# probability ranges
yva_true_rnn, yva_prob_rnn = get_probs_from_loader(cnn_rnn, val_loader)
print("CNN-RNN val prob min/mean/max:", yva_prob_rnn.min(), yva_prob_rnn.mean(), yva_prob_rnn.max())

yva_true_cnn, yva_prob_cnn = get_probs_from_loader(cnn, val_loader)
print("CNN val prob min/mean/max:", yva_prob_cnn.min(), yva_prob_cnn.mean(), yva_prob_cnn.max())

train fraud rate: 0.0018 count: 27 / 15000
val fraud rate: 0.001 count: 15 / 15000
test fraud rate: 0.0018 count: 27 / 15000
clients with >=1 fraud: 296 / 300
CNN-RNN val prob min/mean/max: 0.0037344834 0.06968447 0.74603146
CNN val prob min/mean/max: 0.008381785 0.055328917 0.59090626


In [57]:
# cleanup

del Xtr_seq, Xva_seq, Xte_seq
del ytr_seq, yva_seq, yte_seq
del train_loader, val_loader, test_loader
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()